Date:Feb 21 2023

Author: Aminath Shausan

This program cleans the Twitter (covid and influenza), CRISPER (covid) NNDSS (influenza) data. 


In [1]:
# path = "/Users/uqashaus/OneDrive - The University of Queensland/Git/DigitalDiseaseSurveillance"
path = '/Users/sha481/Documents/UQGitHub/DigitalDiseaseSurveillance'

In [2]:
import pandas as pd
import numpy as np
import re
#from datetime import datetime

# Twitter data
## Data Extraction

Extract required columns from original dataset

In [ ]:
#read original data and scrap required columns
#check the col names of the returned csv
def get_cols(yrname, filename):
    df = pd.read_csv(path + '/data/Twitter/' + yrname + '/' + filename + '.csv', 
                    low_memory=False)
    df_sub = df[['created_at', 'geo.full_name', 'geo.geo.bbox', 'geo.name', 'geo.place_type',]]

    df_sub.columns  
    df_sub.head()

    #save data into csv
    df_sub.to_csv(path + '/data/Twitter/sample/influenza/' + filename + '.csv', index = False) 
    
    return  df_sub #df_sub.head()

In [ ]:
df = get_cols('influenza2020', 'Jan20') 


## Perform basic cleaning 
Format date and full name columns

In [ ]:
def basic_clean(filename):
    df = pd.read_csv(path + '/data/Twitter/sample/influenza/' + filename + '.csv',   
                    low_memory=False) 
    #remove double quoetes
    df = df.replace('"', '', regex=True) 
    #remove everything after T in datetime column
    df['created_at'] = df['created_at'].str.split('T').str[0]
    #change date column to datetime format
    df['created_at'] = pd.to_datetime(df['created_at'], format='%Y-%m-%d')
    #find states 
    df['state'] = df['geo.full_name'].str.split(' ').str[1]
    #rename columns 
    cols = ['Date','GeoFullName','GeoBox', 'GeoName', 'GeoPlaceType', 'State']
    df.columns = cols
    #save df
    df.to_csv(path + '/data/Twitter/sample/clean_influenza/' + filename + '_basic.csv', index = False)
    return df.shape

In [ ]:
df = basic_clean('Jan20')
df

## Check Basic Cleaned dataset 

Note: the State column in the basic clean dataset sometimes contains city names. 
So replace them with their states. 

In [ ]:
# read the basic clean data and check unique names in the State column 
def get_unique(filename):
    df = pd.read_csv(path + '/data/Twitter/sample/clean_influenza/' +  filename + '.csv', 
                    low_memory=False) #,  engine='python' 
    return  df['GeoName'].unique()  #use this with influenza data 
#df['State'].unique() use this with with covid data 

In [ ]:
unique_states = get_unique('June18_basic')
print(unique_states)
type(unique_states) #numpy.ndarray

## Create states from GeoFullName column

In [ ]:
#check if a list of states, cities are in the GeoFullName column


#df_sub = df.head(10)
#df_sub

lst_act = ['Australian Capital Territory' , 'Canberra', 'ACT']
lst_nt= ['Northern Territory' , 'Darwin', 'NT']
lst_nsw = ['New South Wales', 'Sydney']
lst_qld = ['Queensland', 'Brisbane', 'Gold Coast', 'QLD']
lst_vic = ['Victoria', 'Melbourn', 'VIC']
lst_sa = ['South Australia', 'Adelaide', 'SA']
lst_wa = ['Western Australia', 'Perth', 'WA']
lst_ta = ['Tasmania', 'Hobart', 'TAS']

def create_states(filename):
    
    global lst_act, lst_nt, lst_nsw, lst_qld, lst_vic, lst_sa, lst_wa, lst_ta
    df = pd.read_csv(path + '/data/Twitter/sample/clean_influenza/' +filename +
                     '.csv', low_memory=False)
    df['GeoFullName']= df['GeoFullName'].fillna('')
    #df['GeoFullName']=  df['GeoFullName'].str.replace(' ', '')
    geoFullName = df['GeoFullName']
    #print(len(geoFullName))
    df['State']= df['GeoName'].fillna('')
    print(df.shape)
    
    for i in range(0, len(geoFullName)): 
        if any(actName in geoFullName[i] for actName in lst_act):
            df['State'][i]=  'Australian Capital Territory'
        elif any(ntName in geoFullName[i] for ntName in lst_nt):
            df['State'][i]=  'Northern Territory'
        elif any(nswName in geoFullName[i] for nswName in lst_nsw):
            df['State'][i]=  'New South Wales'
        elif any(qldName in geoFullName[i] for qldName in lst_qld):
            df['State'][i]=  'Queensland'
        elif any(vicName in geoFullName[i] for vicName in lst_vic):
            df['State'][i]=  'Victoria'
        elif any(saName in geoFullName[i] for saName in lst_sa):
            df['State'][i]=  'South Australia'
        elif any(waName in geoFullName[i] for waName in lst_wa):
            df['State'][i]=  'Western Australia'
        elif any(taName in geoFullName[i] for taName in lst_ta):
            df['State'][i]=  'Tasmania'
            
    df.to_csv(path + '/data/Twitter/sample/clean_influenza/' + filename + '.csv', index = False)
    return df
    


In [ ]:
df = create_states('Apr20_basic')
df['State'].unique()


In [ ]:
#list of locations and states : 

location = {'Adelaide': 'South Australia',
            'Agnes Water': 'Queensland',
            'Armadale': 'Victoria',
            'Altona North': 'Victoria',
            'Alderley': 'Queensland',
            'Angle Vale': 'South Australia',
            'Ashby': 'New South Wales',
            'Albany': 'Western Australia',
            'Anna Bay': 'New South Wales',
            'Arrawarra': 'New South Wales',
            'Alice Springs': 'Northern Territory',
            'Albury':'New South Wales',
            'Armidale': 'New South Wales',
            'Acton': 'Australian Capital Territory',
            'Bathurst': 'New South Wales',
            'Bardon': 'Queensland',
            'Beaconsfield Upper': 'Victoria',
            'Bunbury': 'Western Australia',
            'Brunswick Heads': 'New South Wales',
            'Bundaberg': 'Queensland', 
            'Benalla': 'Victoria',
            'Berri': 'South Australia',
            'Bellingen': 'New South Wales',
            'Brisbane': 'Queensland',
            'Blue Mountains': 'New South Wales',
            'Blue Mountain Heights': 'New South Wales',
            'Ballarat': 'Victoria',
            'Byron Bay': 'Queensland',
            'Bungendore': 'New South Wales',
            'Bundanoon': 'New South Wales',
            'Balmain': 'New South Wales',
            'Bendigo': 'Victoria',
            'Busselton': 'Western Australia',
            'Barraba': 'New South Wales',
            'Ballina': 'New South Wales', 
            'Blackheath': 'New South Wales',
            'Binalong Bay': 'Tasmania',
            'Balgal Beach': 'Queensland',
            'Bundalong': 'Victoria',
            'Boonah': 'Queensland',
            'Orange': 'New South Wales', 
            'Braddon': 'Australian Capital Territory',
            'Blacktown': 'New South Wales',
            'Beechworth': 'Victoria',
            'Bundoora': 'Victoria',
            'Beerwah': 'Queensland',
            'Bermagui': 'New South Wales',
            'Bargara - Innes Park':'Queensland',
            'Barham': 'New South Wales',
            'Broken Hill': 'New South Wales',
            'Broome': 'Western Australia',
            'Bowral - Mittagong': 'New South Wales',
            'Brunswick East': 'Victoria',
            'Darlinghurst': 'New South Wales',
            'Drouin': 'Victoria',
            'Canberra': 'Australian Capital Territory',
            'Colo Vale': 'New South Wales',
            'Corowa': 'New South Wales',
            'Colac': 'Victoria',
            'Crows Nest': 'Queensland',
            'Central Coast': 'New South Wales',
            'Coffs Harbour': 'New South Wales',
            'Cessnock': 'New South Wales', 
            'Cairns' :'Queensland',
            'Clunes': 'Victoria',
            'Crafers - Bridgewater': 'South Australia',
            'Camperdown': 'New South Wales',
            'Cumbalum': 'New South Wales',
            'Chatswood': 'New South Wales',
            'Carlton' : 'Victoria',
            'Cape Schanck': 'Victoria',
            'Caulfield North': 'Victoria',
            'Cudlee Creek': 'South Australia', 
            'Capalaba': 'Queensland',
            'Cannington': 'Western Australia',
            'Coral Sea': 'Queensland',
            'Cedar Vale': 'Queensland',
            'Castlemaine': 'Victoria', 
            'Churchill': 'Victoria',
            'Dubbo': 'New South Wales',
            'Darwin' : 'Northern Territory',
            'Denman': 'New South Wales',
            'Derby': 'Western Australia',
            'Docklands': 'Victoria', 
            'Devonport': 'Tasmania',
            'Drysdale - Clifton Springs':'Victoria',
            'Dwellingup': 'Western Australia',
            'Deloraine': 'Tasmania',
            'Dee Why': 'New South Wales',
            'Dunsborough': 'Western Australia', 
            'Ellenbrook': 'Western Australia',
            'Evandale': 'Tasmania',
            'East Jindabyne': 'New South Wales',
            'Esk': 'Queensland',
            'Elizabeth Grove': 'South Australia',
            'Empire Bay': 'New South Wales', 
            'Euroa': 'Victoria',
            'Emerald': 'Queensland',
            'Forestville': 'New South Wales',
            'Freeling': 'South Australia',
            'Fraser Island': 'Queensland',
            'Forster - Tuncurry': 'New South Wales',
            'Geraldton': 'Western Australia',
            'Greenbushes': 'Western Australia',
            'Gawler': 'South Australia',
            'Gununa': 'Queensland',
            'Glebe': 'New South Wales',
            'Ghan': 'Northern Territory',
            'Gold Coast': 'Queensland',
            'Gladstone': 'Queensland',
            'Griffith': 'New South Wales',
            'Geelong': 'Victoria',
            'Garden Island Creek': 'Tasmania',
            'Good Night': 'Queensland',
            'Glass House Mountains': 'Queensland',
            'Glen Aplin': 'Queensland',
            'Geeveston': 'Tasmania',
            'Hobart': 'Tasmania',
            'Healesville': 'Victoria',
            'Hamilton': 'Victoria',
            'Highfields':'Queensland',
            'Hervey Bay': 'Queensland',
            'Horsham': 'Victoria',
            'Helensburgh': 'New South Wales',
            'Inverloch': 'Queensland',
            'Innisfail': 'Queensland',
            'Jimboomba - West': 'Queensland',
            'Jimboomba' : 'Queensland', 
            'Joondalup': 'Western Australia',
            'Kooralbyn': 'Queensland',
            'Kirwans Bridge': 'Victoria',
            'Kalgoorlie - Boulder': 'Western Australia',
            'Kenmore' : 'Queensland',
            'Katherine': 'Northern Territory',
            'Kiama': 'New South Wales',
            'Kinka Beach': 'Queensland',
            'Kadina': 'South Australia',
            'Kilmore': 'Victoria',
            'Kojonup': 'Western Australia',
            'Kurri Kurri - Weston': 'New South Wales',
            'Kingston': 'Australian Capital Terrotory',
            'Kingaroy': 'Queensland',
            'Lennox Head':'New South Wales',       
            'Launceston': 'Tasmania',
            'Lismore': 'New South Wales',
            'Lithgow': 'New South Wales',
            'Lorne': 'Victoria',
            'Lake Conjola': 'New Sout Wales',
            'Lara': 'Victoria',
            'Melbourne': 'Victoria',
            'Mackay': 'Queensland',
            'Macedon': 'Victoria',
            'Maryknoll': 'Victoria',
            'McLaren Vale': 'South Australia',
            'Moree': 'New South Wales', 
            'Mooloolaba':'Queensland',
            'Maitland': 'New South Wales',
            'Melton': 'Victoria',
            'Morayfield': 'Queensland',
            'Mosman': 'New South Wales',
            'Margaret River': 'Western Australia',
            'Mount Cotton': 'Queensland', 
            'Murrumbeena': 'Victoria',
            'Murray Bridge': 'South Australia',
            'Merewether': 'New South Wales',
            'Murrurundi': 'New South Wales',
            'Nhulunbuy': 'Northern Terrotory',
            'Mount Hill': 'Western Australia',
            'Marong': 'Victoria',
            'Moranbah': 'Queensland',
            'Mitchell Park': 'Victoria', 
            'Maddens Plains': 'New South Wales',
            'Mount Burnett': 'Victoria',
            'Morwell': 'Victoria', 
            'Merredin': 'Western Australia',
            'Maudsland': 'Queensalnd',
            'Morisset - Cooranbong': 'New South Wales',
            'Macleay Island': 'Queensland',
            'Mataranka': 'Northern Territory',
            'Moss Vale': 'New South Wales',
            'Malua Bay': 'New South Wales',
            'Mia Mia': 'Victoria',
            'Mudgee': 'New South Wales',
            'Mildura': 'Victoria',
            'Newcastle': 'New South Wales',
            'Medowie': 'New South Wales',
            'NSW' : 'New South Wales',
            'Nyah West': 'Victoria',
            'Nelly Bay': 'Queensland',
            'Nerang': 'Queensland',
            'North Strathfield': 'New South Wales',
            'Narooma': 'New South Wales',
            'New Norfolk': 'Tasmania',
            'Norseman': 'Western Australia',
            'North South Australia': 'North South Australia', 
            'Northam': 'Western Australia',
            'Nambour': 'Queensland',
            'Newport': 'Victoria',
            'Newman': 'Western Australia',
            'Narromine': 'New South Wales',
            'Napperby': 'South Australia',
            'Ocean Shores': 'New South Wales',
            'Nelson Bay - Corlette': 'New South Wales',
            'Phillip': 'Australian Capital Territory', 
            'Pakenham': 'Victoria',
            'Perth': 'Western Australia',
            'Port Macquarie': 'New South Wales',
            'Port Vincent': 'South Australia',
            'Port Broughton': 'South Australia',
            'Port Douglas - Craiglie': 'Queensland',
            'Port Victoria': 'Victoria',
            'Port Lincoln': 'South Australia',
            'Port Augusta': 'South Australia',
            'Proserpine': 'Queensland',
            'Port Pirie': 'South Australia',
            'Porepunkah': 'Victoria',
            'Pyrmont': 'New South Wales',
            'Pacific Pines': 'Queensland', 
            'Palmwoods': 'Queensland',
            'Queanbeyan' : 'New South Wales',
            'Queenstown': 'Tasmania',
            'Rockhampton': 'Queensland',
            'Rosebud': 'Victoria',
            'Rottnest Island': 'Western Australia',
            'Redfern': 'New South Wales',
            'Ravensthorpe': 'Western Australia',
            'Rosebery': 'Tasmania',
            'Southern River': 'Western Australia', 
            'Sydney' : 'New South Wales',
            'Snake Valley': 'Victoria',
            'Sunshine Coast': 'Queensland',
            'Surf Beach - Sunderland Bay': 'Victoria',
            'Stanthorpe': 'Queensland',
            'Sandy Bay': 'Tasmania',
            'Strathalbyn': 'South Australia',
            'Silvan': 'Victoria',
            'South Brisbane': 'Queensland',
            'Scone': 'New South Wales', 
            'South Yarra': 'Victoria',
            'Sale': 'Victoria',
            'Summerlands': 'Victoria',
            'Shepparton - Mooroopna': 'Victoria',
            'Shoalhaven Heads': 'New South Wales',
            'St Lucia': 'Queensland',
            'St Marys': 'Tasmania',
            'Stanwell Park': 'New South Wales',
            'Sunbury': 'Victoria',
            'Tumut': 'New South Wales',
            'Two Wells': 'South Australia',
            'Tamworth': 'New South Wales',
            'Tintenbar': 'New South Wales',
            'The Rocks': 'New South Wales',
            'Torquay - Jan Juc': 'Victoria',
            'Townsville': 'Queensland',
            'Tamborine Mountain': 'Queensland',
            'Tweed Heads': 'New South Wales',
            'Thursday Island': 'Queensland',
            'Temora': 'New South Wales',
            'Toowoomba': 'Queensland',
            'The University of South Australia' : 'South Australia',
            'Trafalgar': 'Victoria',
            'Thirlmere': 'New South Wales',
            'Traralgon': 'Victoria',
            'Taree': 'New South Wales',
            'Tatura': 'Victoria',
            'Tahmoor': 'New South Wales',
            'Ulmarra': 'New South Wales',
            'Uralla': 'New South Wales',
            'Ulverstone': 'Tasmania',
            'Ulladulla': 'New South Wales',
            'Victor Harbor - Goolwa': 'South Australia',
            'Warragul': 'Victoria',
            'Warrnambool' : 'Victoria',
            'Wollongong': 'New South Wales',
            'Wagga Wagga': 'New South Wales',
            'Wangaratta': 'Victoria',
            'Whyalla': 'South Australia',
            'Wodonga': 'Victoria',
            'Wingello': 'New South Wales',
            'Wilberforce': 'New South Wales',
            'Wauchope': 'New South Wales',
            'Woori Yallock - Launching Place': 'Victoria',
            'Willyaroo': 'South Australia',
            'Warwick': 'Queensland',
            'Walloon': 'Queensland',
            'Yulara':  'Northern Territory',
            'Yass': 'New South Wales',
            'Yamba': 'New South Wales',
            'Yarrabah': 'Queensland',
            'Yarrawonga': 'Victoria'
           }

#df = pd.read_csv(path + '/data/Twitter/sample/Clean_covid/Aug20_basic.csv', 
#                    low_memory=False) 

In [ ]:
#change names and save df --------- no longer required
def change_name(filename):
    df = pd.read_csv(path + '/data/Twitter/sample/clean_influenza/' +  filename + '.csv', 
                    low_memory=False, skipinitialspace=True) #,  engine='python' 
    df['State'] = df['GeoName'].replace(location, regex=True)
    df['State'] = df['State'].str.replace(r"\(.*\)","", regex=True)
    #save data into csv
    df.to_csv(path + '/data/Twitter/sample/clean_influenza/' + filename + '.csv', index = False) 
    
    return  df['State'].unique()

In [ ]:
newunique_states = change_name('Mar20_basic')
newunique_states

## Group Tweets by weeks

In [55]:
def group_byweek(filename):
    df = pd.read_csv(path + '/data/Twitter/sample/clean_influenza/' +  filename + '.csv',
                     low_memory=False, skipinitialspace=True)
    #format Date column as datetime 
    #df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d') #use this for influenza  
    # df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y') #use this for influenza  
    #extract required columns 
    df_sub = df[['Date', 'State']]
    df_sub = df_sub.copy()  #this avoids warning
    df_sub.loc[:,'Count'] = 1   #add a column with 1s
    #insert weeks
    df_sub.loc[:,'Week'] = df_sub['Date'].dt.to_period('W').dt.to_timestamp()
    #groupby weeks
    df_grp = df_sub.groupby(['State', 'Week'], as_index=False).sum(numeric_only=True)
    return df_grp

In [ ]:
# df = pd.read_csv(path + '/data/Twitter/sample/clean_influenza/Jan18_basic.csv',
#                      low_memory=False, skipinitialspace=True)
# df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y') #use this for influenza
# df_sub = df[['Date', 'State']]
# df_sub = df_sub.copy()  #this avoids warning
# df_sub.loc[:,'Count'] = 1   #add a column with 1s
# # # #insert weeks
# df_sub.loc[:,'Week'] = df_sub['Date'].dt.to_period('W-THU').dt.start_time
# df_grp = df_sub.groupby(['State', 'Week'], as_index=False).sum(numeric_only=True)

# #df_sub.to_csv(path + '/data/Twitter/sample/clean_influenza/df_sub_Week2.csv', index = False)

# df_sub.dtypes

# #df_grp['State'].str.strip()  #--- this doesn't work
# print(df_grp['State'].unique())
# df_grp.loc[df_grp['State'] == 'Queensland']

## Prepare dataframes for each State

In [44]:
df_ACT = pd.DataFrame(columns=['State', 'Week', 'Count'])
df_NSW = pd.DataFrame(columns=['State', 'Week', 'Count'])
df_NT = pd.DataFrame(columns=['State', 'Week', 'Count'])
df_QLD= pd.DataFrame(columns=['State', 'Week', 'Count'])
df_SA = pd.DataFrame(columns=['State', 'Week', 'Count'])
df_TAS = pd.DataFrame(columns=['State', 'Week', 'Count'])
df_VIC= pd.DataFrame(columns=['State', 'Week', 'Count'])
df_WA = pd.DataFrame(columns=['State', 'Week', 'Count'])
df_WA

,State,Week,Count


In [45]:
def get_state_df(filename):
    
    global df_ACT, df_NSW, df_NT, df_QLD, df_SA, df_TAS, df_VIC, df_WA
    #get the grouped by week  data
    df_grp = group_byweek(filename)
     
    
    #filter states
    df_ACT_sub = df_grp.loc[df_grp['State'] == 'Australian Capital Territory']
    df_NSW_sub = df_grp.loc[df_grp['State'] == 'New South Wales']
    df_NT_sub = df_grp.loc[df_grp['State'] == 'Northern Territory']
    df_QLD_sub = df_grp.loc[df_grp['State'] == 'Queensland']
    df_SA_sub = df_grp.loc[df_grp['State'] == 'South Australia']
    df_TAS_sub = df_grp.loc[df_grp['State'] == 'Tasmania']
    df_VIC_sub = df_grp.loc[df_grp['State'] == 'Victoria']
    df_WA_sub = df_grp.loc[df_grp['State'] == 'Western Australia']
    
    #add rows to end of dfs already created 
    if not df_ACT_sub.empty:
           df_ACT = pd.concat([df_ACT, df_ACT_sub], ignore_index = True)
    if not df_NSW_sub.empty:
           df_NSW = pd.concat([df_NSW, df_NSW_sub], ignore_index = True)
    if not df_NT_sub.empty:
           df_NT = pd.concat([df_NT, df_NT_sub], ignore_index = True)
    if not df_QLD_sub.empty:
           df_QLD = pd.concat([df_QLD, df_QLD_sub], ignore_index = True)
    if not df_SA_sub.empty:
           df_SA = pd.concat([df_SA, df_SA_sub], ignore_index = True)
    if not df_TAS_sub.empty:
           df_TAS = pd.concat([df_TAS, df_TAS_sub], ignore_index = True)
    if not df_VIC_sub.empty:
           df_VIC = pd.concat([df_VIC, df_VIC_sub], ignore_index = True)
    if not df_WA_sub.empty:
           df_WA = pd.concat([df_WA, df_WA_sub], ignore_index = True)
    
    #save data into csv
    df_ACT.to_csv(path + '/data/Twitter/sample/clean_influenza/df_ACT.csv', index = False)
    df_NSW.to_csv(path + '/data/Twitter/sample/clean_influenza/df_NSW.csv', index = False)
    df_NT.to_csv(path + '/data/Twitter/sample/clean_influenza/df_NT.csv', index = False)
    df_QLD.to_csv(path + '/data/Twitter/sample/clean_influenza/df_QLD.csv', index = False)
    df_SA.to_csv(path + '/data/Twitter/sample/clean_influenza/df_SA.csv', index = False)
    df_TAS.to_csv(path + '/data/Twitter/sample/clean_influenza/df_TAS.csv', index = False)
    df_VIC.to_csv(path + '/data/Twitter/sample/clean_influenza/df_VIC.csv', index = False)
    df_WA.to_csv(path + '/data/Twitter/sample/clean_influenza/df_WA.csv', index = False)
    
    return  df_ACT.shape, df_NSW.shape, df_NT.shape,  df_QLD.shape, df_SA.shape, df_TAS.shape, df_VIC.shape, df_WA.shape

In [51]:

df= get_state_df('Mar18_basic')
df

((5, 3), (14, 3), (1, 3), (13, 3), (11, 3), (4, 3), (14, 3), (11, 3))

Note: the above grouping creates dfs with repeated dates. 
So merge those rows and add the counts for those rows

In [111]:
filename = 'df_WA'
df = pd.read_csv(path + '/data/Twitter/sample/clean_influenza/' +  filename + '.csv',
                     low_memory=False, parse_dates = ['Week'], dayfirst =True)
#     #format Date column as datetime 
# df['Week'] = pd.to_datetime(df['Week'], format='%d/%m/%y')
# df['Week'] = pd.to_datetime(df['Week'], format='%Y-%m-%d') #use this for influenza  
print(df.dtypes)
print(df.shape)
print(df['Week'].is_unique)
df

# df2 = df.copy()  #this avoids warning
df2 = df.groupby(['State','Week',],as_index=False).sum(numeric_only=True) #numeric_only=True
print(df2['Week'].is_unique)  #check if Week column has  unique values
print(df2.shape)
df2.to_csv(path + '/data/Twitter/sample/clean_influenza/' +filename + '.csv', index = False)
df2

State            object
Week     datetime64[ns]
Count             int64
dtype: object
(258, 3)
False
True
(237, 3)


,State,Week,Count
0,Western Australia,2018-01-01,1
1,Western Australia,2018-01-08,3
2,Western Australia,2018-01-15,1
3,Western Australia,2018-01-29,3
4,Western Australia,2018-02-05,2
...,...,...,...
232,Western Australia,2022-11-28,2
233,Western Australia,2022-12-05,9
234,Western Australia,2022-12-12,2
235,Western Australia,2022-12-19,5


# NNDSS Influenza data 

- Original Data is recorded at the end of the week on Fridays. 
- Data for ACT is not included 
- Data is available upto end of 2021
- data from: https://www.health.gov.au/resources/publications/national-notifiable-diseases-surveillance-system-nndss-public-dataset-influenza-laboratory-confirmed 

In [7]:
# load data and 
df = pd.read_csv(path + '/data/NNDSS_influenza/influenza_data.csv',
                     low_memory=False, skipinitialspace=True)
#print(df['Week'][0])#day/month/year 
#change week to date format 
df['Week'] = pd.to_datetime(df['Week'], format='%d/%m/%Y') #use this for influenza

df.loc[:,'Count'] = 1   #add a column with 1s
#  #format week so that it starts from monday of week
df.loc[:,'newWeek'] = df['Week'].dt.to_period('W').dt.to_timestamp()
# df.to_csv(path + '/data/NNDSS_influenza/influenza_data_newWeek.csv', index = False)
df_grp = df.groupby(['State', 'newWeek'], as_index=False).sum(numeric_only=True)
# #df_grp['State'].unique()  #['NSW', 'Qld', 'SA', 'Vic', 'WA', 'NT', 'Tas']
# #df_grp
# #df.dtypes
df_grp
# df

,State,newWeek,Count
0,NSW,2016-02-15,141
1,NSW,2016-02-22,149
2,NSW,2016-02-29,173
3,NSW,2016-03-07,173
4,NSW,2016-03-14,152
...,...,...,...
1917,WA,2021-11-08,2
1918,WA,2021-11-22,2
1919,WA,2021-12-06,2
1920,WA,2021-12-13,1


In [22]:
###check missing instrances
check = df_grp[(df_grp['newWeek']<= '2019-12-23') & (df_grp['newWeek']>='2018-01-01')]  ## should have 104 rows 
check ## 730 rows
check.loc[check['State'] == 'NSW'] #104
check.loc[check['State'] == 'NT'] #99, missing 5
check.loc[check['State'] == 'Qld'] #104
check.loc[check['State'] == 'SA']##104
check.loc[check['State'] == 'Tas']##104
check.loc[check['State'] == 'Vic']##104
check.loc[check['State'] == 'WA']##104

,State,newWeek,Count
1747,WA,2018-01-01,53
1748,WA,2018-01-08,52
1749,WA,2018-01-15,67
1750,WA,2018-01-22,66
1751,WA,2018-01-29,97
...,...,...,...
1846,WA,2019-11-25,51
1847,WA,2019-12-02,55
1848,WA,2019-12-09,69
1849,WA,2019-12-16,42


In [ ]:
def get_state_infldf():
    
    global df_grp, df_NSW, df_NT, df_QLD, df_SA, df_TAS, df_VIC, df_WA
        
    
    #filter states
    df_NSW_sub = df_grp.loc[df_grp['State'] == 'NSW']
    df_NT_sub = df_grp.loc[df_grp['State'] == 'NT']
    df_QLD_sub = df_grp.loc[df_grp['State'] == 'Qld']
    df_SA_sub = df_grp.loc[df_grp['State'] == 'SA']
    df_TAS_sub = df_grp.loc[df_grp['State'] == 'Tas']
    df_VIC_sub = df_grp.loc[df_grp['State'] == 'Vic']
    df_WA_sub = df_grp.loc[df_grp['State'] == 'WA']
    
    #add rows to end of dfs already created 
    if not df_NSW_sub.empty:
           df_NSW = pd.concat([df_NSW, df_NSW_sub], ignore_index = True)
    if not df_NT_sub.empty:
           df_NT = pd.concat([df_NT, df_NT_sub], ignore_index = True)
    if not df_QLD_sub.empty:
           df_QLD = pd.concat([df_QLD, df_QLD_sub], ignore_index = True)
    if not df_SA_sub.empty:
           df_SA = pd.concat([df_SA, df_SA_sub], ignore_index = True)
    if not df_TAS_sub.empty:
           df_TAS = pd.concat([df_TAS, df_TAS_sub], ignore_index = True)
    if not df_VIC_sub.empty:
           df_VIC = pd.concat([df_VIC, df_VIC_sub], ignore_index = True)
    if not df_WA_sub.empty:
           df_WA = pd.concat([df_WA, df_WA_sub], ignore_index = True)
    
    #save data into csv
    df_NSW.to_csv(path + '/data/NNDSS_influenza/df_NSW.csv', index = False)
    df_NT.to_csv(path + '/data/NNDSS_influenza/df_NT.csv', index = False)
    df_QLD.to_csv(path + '/data/NNDSS_influenza/df_QLD.csv', index = False)
    df_SA.to_csv(path + '/data/NNDSS_influenza/df_SA.csv', index = False)
    df_TAS.to_csv(path + '/data/NNDSS_influenza/df_TAS.csv', index = False)
    df_VIC.to_csv(path + '/data/NNDSS_influenza/df_VIC.csv', index = False)
    df_WA.to_csv(path + '/data/NNDSS_influenza/df_WA.csv', index = False)
    
    return   df_NSW.shape, df_NT.shape,  df_QLD.shape, df_SA.shape, df_TAS.shape, df_VIC.shape, df_WA.shape

In [ ]:
df= get_state_infldf()
df

## Get weekly nnds data from all states 

In [4]:
df = pd.read_csv(path + '/data/NNDSS_influenza/influenza_data_newWeek.csv',
                     low_memory=False, skipinitialspace=True)
df.head()
#print(df['Week'][0])#day/month/year 
df_grp = df.groupby(['newWeek'], as_index=False).sum(numeric_only=True)
df_grp.head()

,newWeek,Count
0,1/1/2018,662
1,1/10/2018,2297
2,1/11/2021,9
3,1/2/2021,13
4,1/3/2021,13


# CRISPER Covid data 

In [ ]:
df = pd.read_csv(path + '/data/Crisper/cases/covid19data_au_state_cases_new.csv',
                     low_memory=False, skipinitialspace=True)
# print(df['date'][0])#day/month/year 
# print(type(df['date'][0]))

#change week to date format 
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y') 
df.loc[:,'week'] = df['date'].dt.to_period('W').dt.to_timestamp()
df['state'].unique() #['WA', 'VIC', 'TAS', 'SA', 'QLD', 'NT', 'NSW', 'ACT']
df.to_csv(path + '/data/Crisper/covid19cases_week.csv', index = False)
df_grp = df.groupby(['state', 'week'], as_index=False).sum(numeric_only=True)
#df

In [ ]:
df_ACT = pd.DataFrame(columns=['state', 'week', 'cases'])
df_NSW = pd.DataFrame(columns=['state', 'week', 'cases'])
df_NT = pd.DataFrame(columns=['state',  'week', 'cases'])
df_QLD= pd.DataFrame(columns=['state',  'week', 'cases'])
df_SA = pd.DataFrame(columns=['state',  'week', 'cases'])
df_TAS = pd.DataFrame(columns=['state',  'week', 'cases'])
df_VIC= pd.DataFrame(columns=['state',  'week', 'cases'])
df_WA = pd.DataFrame(columns=['state',  'week', 'cases'])

In [ ]:
def get_state_covid_df():
    
    global df_grp, df_ACT, df_NSW, df_NT, df_QLD, df_SA, df_TAS, df_VIC, df_WA
        
  
    #filter states
    df_ACT_sub = df_grp.loc[df_grp['state'] == 'ACT']
    df_NSW_sub = df_grp.loc[df_grp['state'] == 'NSW']
    df_NT_sub = df_grp.loc[df_grp['state'] == 'NT']
    df_QLD_sub = df_grp.loc[df_grp['state'] == 'QLD']
    df_SA_sub = df_grp.loc[df_grp['state'] == 'SA']
    df_TAS_sub = df_grp.loc[df_grp['state'] == 'TAS']
    df_VIC_sub = df_grp.loc[df_grp['state'] == 'VIC']
    df_WA_sub = df_grp.loc[df_grp['state'] == 'WA']
    
    #add rows to end of dfs already created 
    if not df_ACT_sub.empty:
           df_ACT = pd.concat([df_ACT, df_ACT_sub], ignore_index = True)
    if not df_NSW_sub.empty:
           df_NSW = pd.concat([df_NSW, df_NSW_sub], ignore_index = True)
    if not df_NT_sub.empty:
           df_NT = pd.concat([df_NT, df_NT_sub], ignore_index = True)
    if not df_QLD_sub.empty:
           df_QLD = pd.concat([df_QLD, df_QLD_sub], ignore_index = True)
    if not df_SA_sub.empty:
           df_SA = pd.concat([df_SA, df_SA_sub], ignore_index = True)
    if not df_TAS_sub.empty:
           df_TAS = pd.concat([df_TAS, df_TAS_sub], ignore_index = True)
    if not df_VIC_sub.empty:
           df_VIC = pd.concat([df_VIC, df_VIC_sub], ignore_index = True)
    if not df_WA_sub.empty:
           df_WA = pd.concat([df_WA, df_WA_sub], ignore_index = True)
    
    #save data into csv
    df_ACT.to_csv(path + '/data/Crisper/cases/df_ACT.csv', index = False)
    df_NSW.to_csv(path + '/data/Crisper/cases/df_NSW.csv', index = False)
    df_NT.to_csv(path + '/data/Crisper/cases/df_NT.csv', index = False)
    df_QLD.to_csv(path + '/data/Crisper/cases/df_QLD.csv', index = False)
    df_SA.to_csv(path + '/data/Crisper/cases/df_SA.csv', index = False)
    df_TAS.to_csv(path + '/data/Crisper/cases/df_TAS.csv', index = False)
    df_VIC.to_csv(path + '/data/Crisper/cases/df_VIC.csv', index = False)
    df_WA.to_csv(path + '/data/Crisper/cases/df_WA.csv', index = False)
    
    return   df_ACT.shape, df_NSW.shape, df_NT.shape,  df_QLD.shape, df_SA.shape, df_TAS.shape, df_VIC.shape, df_WA.shape

In [ ]:
df= get_state_covid_df()
df

## Google Trends Data

In [26]:
df = pd.read_csv(path + '/data/GoogleTrends/AU.csv',
                     low_memory=False)  #, parse_dates=['date']
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d') #format='%d/%m/%y'
#  #format week so that it starts from monday of week
df.loc[:,'Week'] = df['date'].dt.to_period('W').dt.to_timestamp()
df.to_csv(path + '/data/GoogleTrends/df_AU.csv', index = False)

print(df['date'].dtypes)
df

datetime64[ns]


,date,coronavirus,covid,influenza,Haemophilus influenzae,flu,parainfluenza,H1N1,H7N9,H5N1,...,metapneumovirus,Bordetella pertussis,Mycoplasma pneumoniae,pneumonia,bronchitis,H9N2,sinusitis,upper respiratory tract infection,Tamiflu,Week
0,2018-01-07,0,0,7,26,6,0,0,76,10,...,0,0,0,18,39,0,61,0,7,2018-01-01
1,2018-01-14,0,0,6,0,5,17,0,46,0,...,8,0,31,15,26,0,44,64,14,2018-01-08
2,2018-01-21,0,0,7,70,5,0,0,0,0,...,0,0,0,15,17,0,32,24,13,2018-01-15
3,2018-01-28,0,0,9,0,6,0,0,0,0,...,0,0,0,19,29,0,34,80,9,2018-01-22
4,2018-02-04,0,0,8,0,5,0,0,0,10,...,0,0,0,16,31,0,34,69,18,2018-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2022-12-04,0,8,7,20,6,25,0,0,6,...,40,40,0,23,45,23,60,19,8,2022-11-28
257,2022-12-11,0,8,8,0,7,29,2,0,0,...,15,0,0,22,56,0,69,0,4,2022-12-05
258,2022-12-18,0,9,10,0,7,32,5,0,19,...,44,0,23,23,50,0,56,26,0,2022-12-12
259,2022-12-25,0,7,8,0,6,0,3,0,0,...,34,0,0,20,42,0,56,0,12,2022-12-19


In [ ]:
print(df['date'][0])
#df.shape
df.dtypes


#### Date: 04 Sept 2023 
## Prepare NNDSS and Google Trends data for spatiotemporal modelling 

In [50]:
#define a function to read GT data and insert state
def read_data(filename, statename):
    df = pd.read_csv(path + '/data/GoogleTrends/' + filename + '.csv')
#drop unwanted columns. The unwanted columns are for covid search terms 
    df = df.drop(['coronavirus', 'covid'], axis=1)
    print(df.shape) #(261, 23)
    df.insert(1,'State',statename)  #add state 
     # df_agg.insert(0,'SA3Code',df_agg['SA3Region'].replace(region_to_sa3_mapping))
    print(df['State'].unique())
    # print(df.dtypes) #date is in object format
    df.head()
    
    return df

# filename = 'AU-ACT'
df_act = read_data('AU-ACT', 'ACT')
df_nsw = read_data('AU-NSW', 'NSW')
df_nt = read_data('AU-NT', 'NT') #no data for 'flu' and 'sinusitis'
df_qld = read_data('AU-QLD', 'QLD')
df_sa = read_data('AU-SA', 'SA')
df_tas = read_data('AU-TAS', 'TAS')
df_vic = read_data('AU-VIC', 'VIC')
df_wa = read_data('AU-WA', 'WA')


df_nt.head() 

(261, 23)
['ACT']
(261, 23)
['NSW']
(261, 21)
['NT']
(261, 23)
['QLD']
(261, 23)
['SA']
(261, 23)
['TAS']
(261, 23)
['VIC']
(261, 23)
['WA']


,date,State,influenza,Haemophilus influenzae,parainfluenza,H1N1,H7N9,H5N1,H3N2,grippe,...,rhinovirus,respiratory syncytial virus,metapneumovirus,Mycoplasma pneumoniae,pneumonia,Bordetella pertussis,bronchitis,H9N2,upper respiratory tract infection,Tamiflu
0,2018-01-07,NT,29,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-01-14,NT,0,0,0,43,0,0,0,0,...,0,0,0,0,0,0,28,0,0,0
2,2018-01-21,NT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,36,0,0,0
3,2018-01-28,NT,37,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2018-02-04,NT,0,0,0,0,0,0,0,0,...,29,0,0,0,18,0,0,0,0,0


In [51]:
#vertically concatenate dfs 
df_gt =  pd.concat([df_act, df_nsw, df_nt, df_qld, df_sa, df_tas, df_vic, df_wa], axis=0)
df_gt['date'] = pd.to_datetime(df_gt['date'], dayfirst = True)
df_gt.insert(1,'Year', df_gt['date'].dt.strftime('%Y')) #add year column 
df_gt['Year'] = df_gt['Year'].astype('int')
print(df_gt.shape) #(2088, 24) , 261*8 = 2088
print(df_gt['State'].unique())
#save gt data 
# df_gt.to_csv(path + '/data/GoogleTrends/df_gt_all.csv', index=False)


(2088, 24)
['ACT' 'NSW' 'NT' 'QLD' 'SA' 'TAS' 'VIC' 'WA']


In [57]:
df_gt.to_csv(path + '/data/GoogleTrends/df_gt_all.csv', index=False)

print(df_gt.dtypes)
df_gt.head()

date                                 datetime64[ns]
Year                                          int64
State                                        object
influenza                                     int64
Haemophilus influenzae                        int64
flu                                         float64
parainfluenza                                 int64
H1N1                                          int64
H7N9                                          int64
H5N1                                          int64
H3N2                                          int64
grippe                                        int64
gripe                                         int64
adenovirus                                    int64
rhinovirus                                    int64
respiratory syncytial virus                   int64
metapneumovirus                               int64
Bordetella pertussis                          int64
Mycoplasma pneumoniae                         int64
pneumonia   

,date,Year,State,influenza,Haemophilus influenzae,flu,parainfluenza,H1N1,H7N9,H5N1,...,respiratory syncytial virus,metapneumovirus,Bordetella pertussis,Mycoplasma pneumoniae,pneumonia,bronchitis,H9N2,sinusitis,upper respiratory tract infection,Tamiflu
0,2018-01-07,2018,ACT,0,0,4.0,0,100,28,0,...,0,0,0,0,0,0,0,54.0,0,0
1,2018-01-14,2018,ACT,19,0,6.0,20,0,0,24,...,0,0,0,0,0,0,0,19.0,0,92
2,2018-01-21,2018,ACT,0,0,8.0,0,0,32,0,...,0,0,0,0,0,0,0,0.0,0,90
3,2018-01-28,2018,ACT,0,0,11.0,0,36,0,0,...,0,0,0,0,0,0,0,0.0,0,0
4,2018-02-04,2018,ACT,0,0,3.0,0,0,41,0,...,0,21,0,0,0,0,0,0.0,0,0


#### Read Formatted Flu data for all states from NNDSS file

In [32]:
#Note: ACT is not in this dataset, and data is available up to end of 2021
df_nndss = pd.read_csv(path + '/data/NNDSS_influenza/influenza_data_newWeek.csv')
print(df_nndss.shape)#(724919, 4)
df_nndss['Week'] = pd.to_datetime(df_nndss['Week'], dayfirst=True) #convert to datetime
df_nndss = df_nndss[~(df_nndss['Week'] < '2018-01-01')]
print(df_nndss.shape)#(389682, 4)
print(df_nndss.dtypes)
print(df_nndss['State'].unique())
df_nndss.head()

(724919, 4)
(389682, 4)
Week       datetime64[ns]
State              object
Count               int64
newWeek            object
dtype: object
['NSW' 'NT' 'QLD' 'SA' 'TAS' 'VIC' 'WA']


,Week,State,Count,newWeek
335237,2018-01-05,NSW,1,1/1/2018
335238,2018-01-05,NSW,1,1/1/2018
335239,2018-01-05,NSW,1,1/1/2018
335240,2018-01-05,NSW,1,1/1/2018
335241,2018-01-05,NSW,1,1/1/2018


In [33]:
#aggregate count based on date
df_agg = df_nndss.groupby(['Week','State']).agg(
         totalCount = ('Count','sum')).reset_index()
print(df_agg.shape) #1241, 3, shape of df_gt = (1827, 24), #nndss data is available 2021-12-31

df_agg['Year'] = df_agg['Week'].dt.strftime('%Y') #add year column
df_agg.head()


#save flu 
df_agg.to_csv(path + '/data/NNDSS_influenza/df_nndss_all.csv', index=False)
# #check data
# df_agg = pd.read_csv(path + '/data/NNDSS_influenza/df_nndss_all.csv')
# df_agg.head()

(1241, 3)


,Week,State,totalCount,Year
0,2018-01-05,NSW,182,2018
1,2018-01-05,NT,7,2018
2,2018-01-05,QLD,199,2018
3,2018-01-05,SA,85,2018
4,2018-01-05,TAS,2,2018


In [35]:
#read population data 
df_pop = pd.read_csv(path + '/data/population/state_territory_popn.csv')
df_pop= df_pop.drop(['2022'], axis=1)
df_pop = pd.melt(df_pop, id_vars=["State"], value_vars=["2018", "2019","2020", "2021"],
           var_name="Year", value_name="Popn") #convert to long form
 
print(df_pop.dtypes)
print(df_pop.shape) #68 by 3
#8 states, 4 yrs = 32
# df_pop.to_csv(path + '/data/population/df_pop.csv', index=False)

df_pop.head()

#merge df_agg and df_pop
# df_agg.merge(df_pop, how='left', on=['State', 'Year'])
df_merged = df_agg.merge(df_pop, left_on=['State', 'Year'], right_on = ['State', 'Year'],
                         how='left')

print(df_merged.shape)
# df_merged.to_csv(path + '/data/NNDSS_influenza/df_merged.csv', index=False)
df_merged.head()

State    object
Year     object
Popn      int64
dtype: object
(32, 3)


,State,Year,Popn
0,NSW,2018,8003564
1,VIC,2018,6479695
2,QLD,2018,5046434
3,SA,2018,1755715
4,WA,2018,2636404


In [42]:

df_agg.shape #(1241, 4)
print([df_agg['State'].unique(), df_agg['Year'].unique()])
print([df_pop['State'].unique(), df_pop['Year'].unique()])
print([df_merged['State'].unique(), df_merged['Year'].unique()])

[array(['NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA'], dtype=object), array(['2018', '2019', '2020', '2021'], dtype=object)]
[array(['NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT'], dtype=object), array(['2018', '2019', '2020', '2021'], dtype=object)]
[array(['NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA'], dtype=object), array(['2018', '2019', '2020', '2021'], dtype=object)]


In [ ]:
#summarise flu data
print(df_agg['totalCount'].agg(['min', 'max','mean','std']))
# min        1.000000
# max     8652.000000
# mean     314.006446
# std      855.257851

#for nndss, data is from 2018 to 2021
print(min(df_nndss['Week'].unique())) #2018-01-05 Friday
print(max(df_nndss['Week'].unique())) #2021-12-31, Friday
#for ggooglr trends, data is from 2018 to 202
print(min(df_gt['date'].unique())) #2018-01-07, saturday
print(max(df_gt['date'].unique())) #2023-01-01, Wednesday 

#### Combine googleTrend and flu data

Add a date column to df_merged (which is flu data) by adding two days to each element of 
Week column. This would align flu dates to gt dates as Gt data are recorded on Saturday of the week and flu data are recorded on Friday of the week.



In [60]:
df_merged.head()

,Week,State,totalCount,Year,Popn
0,2018-01-05,NSW,182,2018,8003564
1,2018-01-05,NT,7,2018,245920
2,2018-01-05,QLD,199,2018,5046434
3,2018-01-05,SA,85,2018,1755715
4,2018-01-05,TAS,2,2018,542927


In [62]:
# df_merged['date'] = df_merged['Week'] + pd.DateOffset(days=2)
df_merged.insert(1,'date', df_merged['Week']+ pd.DateOffset(days=2))

df_merged.head()

# #check difference between two dates 
# diff = (df_merged['date'] - df_merged['Week']).dt.days
# diff
# diff.unique() #array([2])

,Week,date,State,totalCount,Year,Popn
0,2018-01-05,2018-01-07,NSW,182,2018,8003564
1,2018-01-05,2018-01-07,NT,7,2018,245920
2,2018-01-05,2018-01-07,QLD,199,2018,5046434
3,2018-01-05,2018-01-07,SA,85,2018,1755715
4,2018-01-05,2018-01-07,TAS,2,2018,542927


In [65]:
df_gt.head()

,date,Year,State,influenza,Haemophilus influenzae,flu,parainfluenza,H1N1,H7N9,H5N1,...,respiratory syncytial virus,metapneumovirus,Bordetella pertussis,Mycoplasma pneumoniae,pneumonia,bronchitis,H9N2,sinusitis,upper respiratory tract infection,Tamiflu
0,2018-01-07,2018,ACT,0,0,4.0,0,100,28,0,...,0,0,0,0,0,0,0,54.0,0,0
1,2018-01-14,2018,ACT,19,0,6.0,20,0,0,24,...,0,0,0,0,0,0,0,19.0,0,92
2,2018-01-21,2018,ACT,0,0,8.0,0,0,32,0,...,0,0,0,0,0,0,0,0.0,0,90
3,2018-01-28,2018,ACT,0,0,11.0,0,36,0,0,...,0,0,0,0,0,0,0,0.0,0,0
4,2018-02-04,2018,ACT,0,0,3.0,0,0,41,0,...,0,21,0,0,0,0,0,0.0,0,0


In [67]:
#combine df_merge and df_gt
# df_agg.merge(df_pop, how='left', on=['State', 'Year'])
df_gt_flu = df_gt.merge(df_merged, left_on=['date', 'State'], right_on = ['date', 'State'],
                         how='left')

print(df_gt_flu.shape)

df_gt_flu.head()

(2088, 29)


,date,Year_x,State,influenza,Haemophilus influenzae,flu,parainfluenza,H1N1,H7N9,H5N1,...,pneumonia,bronchitis,H9N2,sinusitis,upper respiratory tract infection,Tamiflu,Week,totalCount,Year_y,Popn
0,2018-01-07,2018,ACT,0,0,4.0,0,100,28,0,...,0,0,0,54.0,0,0,NaT,NaN,NaN,NaN
1,2018-01-14,2018,ACT,19,0,6.0,20,0,0,24,...,0,0,0,19.0,0,92,NaT,NaN,NaN,NaN
2,2018-01-21,2018,ACT,0,0,8.0,0,0,32,0,...,0,0,0,0.0,0,90,NaT,NaN,NaN,NaN
3,2018-01-28,2018,ACT,0,0,11.0,0,36,0,0,...,0,0,0,0.0,0,0,NaT,NaN,NaN,NaN
4,2018-02-04,2018,ACT,0,0,3.0,0,0,41,0,...,0,0,0,0.0,0,0,NaT,NaN,NaN,NaN


In [69]:
df_gt_flu.to_csv(path + '/data/combined/gt_flu.csv', index=False)